In [88]:
# General packages

import requests
import pandas as pd
from bs4 import BeautifulSoup
import numpy as np
import time
import json
import re
from datetime import datetime


In [49]:
# Selenium

from selenium.webdriver.edge import service
from selenium import webdriver
from selenium.webdriver.common.by import By
from collections import Counter
from nltk.corpus import stopwords
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait

# Collect penetration dates from the forum

In [ ]:
#Load data:

pages_pd = pd.read_csv('scraped_product_pages_extracted.csv', sep=";", index_col = "Unnamed: 0") 

For the ones with a github page:

In [ ]:
pages_penetrated = pages[(pages["nonzero_price"]) & (pages["open_issues"].isnull()==False)]

In [ ]:
pages_penetrated["search_query"]=pages_penetrated["githublink"].str.split('/').str[4]

In [ ]:
pages_penetrated_dict = pages_penetrated[["url", "search_query", "main_developer_username", "launch_date"]].to_dict(orient="records")

In [ ]:
#setting up selenium to function
edgeOption = webdriver.EdgeOptions()
edgeOption.use_chromium = True
edgeOption.add_argument("start-maximized")
edgeOption.binary_location = r"C:\Program Files (x86)\Microsoft\Edge\Application\msedge.exe"
s=service.Service(r'C:\Users\Yana Gerina\Documents\msedgedriver.exe')
driver = webdriver.Edge(service=s, options=edgeOption)

In [ ]:
for page in pages_penetrated_dict:
    driver.get("https://forum.xda-developers.com" + page["url"])
    time.sleep(3)
    #Collect the number of open and closed issues if it's available:
    for button in driver.find_elements(by=By.CLASS_NAME, value = "button-text"):
        if button.text == "Search This thread":
            button.click()
            time.sleep(2)
    input_row = driver.switch_to.active_element
    input_row.send_keys("pro |purchase|donation")
    time.sleep(2)
    driver.find_element(by=By.CLASS_NAME, value = "quickSearch-ResultCloser").click()
    time.sleep(1)
    input_row.send_keys(Keys.TAB + Keys.TAB)
    time.sleep(2)
    member_input_row = driver.switch_to.active_element
    member_input_row.send_keys(page["main_developer_username"])
    time.sleep(2)
    member_input_row.send_keys(Keys.ENTER + Keys.ENTER)
    time.sleep(2)
    
    results  = []
    for result in driver.find_elements(by=By.CLASS_NAME, value = "contentRow-main"):
        title = result.find_element(by=By.CLASS_NAME, value = "contentRow-title")
        if re.search("Post", title.text) is not None:
            datestamp = result.find_element(by=By.CLASS_NAME, value = "u-dt").text
            post_number = [x.text for x in result.find_elements(by=By.TAG_NAME, value = "li") if re.search("Post", x.text) is not None][0].replace(",","")
            post_number = int(re.search(r"[\d,]+", post_number).group(0))
            mention = {"title": title.text, "datestamp": datestamp, "post_number": post_number}
            results.append(mention)

        if results:
            earliest_post = min(results, key=lambda x:x['post_number'])
            page["penetration_datestamp"] = earliest_post["datestamp"]
        else: 
            page["penetration_datestamp"] = np.nan
driver.quit()   

In [ ]:
with open('pages_github_penetrated_datestamps.json', 'w') as fp:
    json.dump(pages_penetrated_dict, fp)

## For all non-zero priced products:

In [225]:
# Load clean data:
pages = pd.read_csv('data/safe_obs_dataset.csv', sep=";", index_col = "Unnamed: 0") 

In [36]:
pages

,url,title,launch_date,launch_message,external_links,main_developer_username,main_developer_page,main_developer_title,main_developer_joined,main_developer_messages,...,contrlink_1,contrlink_2,contrlink_3,contrlink_4,contrlink_5,contrlink_6,contrlink_7,number_of_developers,nonzero_price,ext_links_ind
0,/t/app-6-0-compass-of-chance.4440211/,[APP][6.0+]Compass of Chance,"May 2, 2022","\nThe compass does not point to the north, but...",['https://play.google.com/store/apps/details?i...,Baiumka,/m/baiumka.12120441/,New member,"May 2, 2022",4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,False,True
2,/t/app-all-root-solutions-6-0-viper4android-fx...,[APP][ALL ROOT SOLUTIONS][6.0+]ViPER4Android F...,"Apr 7, 2018",\nViPER4Android FX\n\n\n\n\n\nBy pittvandewitt...,['https://labs.xda-developers.com/store/app/co...,Team_DeWitt,/m/team_dewitt.9073985/,Senior Member,"Apr 7, 2018",71,...,https://forum.xda-developers.com/member.php?u=...,https://forum.xda-developers.com/member.php?u=...,https://forum.xda-developers.com/member.php?u=...,NaN,NaN,NaN,NaN,3.0,True,True
4,/t/app-6-0-netguard-no-root-firewall.3233012/,[APP][6.0+] NetGuard - No-root firewall,"Oct 25, 2015",\n\n\n\n\n\n\nNetGuard provides simple and adv...,['https://contact.faircode.eu/?product=netguar...,M66B,/m/m66b.2799345/,Recognized Developer,"Aug 1, 2010","25,652",...,https://forum.xda-developers.com/member.php?u=...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,True,True
7,/t/app-2-2-root-wifi-reaver-gui-for-android.24...,[APP][2.2+][ROOT][WiFi] Reaver-GUI for Android,"Sep 24, 2013",\n\n\n\n\n\nReaver for Android v1.30\nReaver-W...,['https://bcmon.googlecode.com/files/bcmon.apk...,SOEDI,/m/soedi.4096618/,Senior Member,"May 14, 2011",154,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,False,True
8,/t/app-11-dual-camera-video-recorder.4447581/,[APP][11+] Dual Camera Video Recorder,Today at 8:08 AM,\nCapture video from rear and front cameras at...,['https://play.google.com/store/apps/details?i...,User848,/m/user848.5214037/,Senior Member,"Apr 9, 2013",78,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38406,/t/free-apps-packet-good-apps-packet.678403/,[FREE APPS PACKET] Good apps packet.,"May 9, 2010",\neeehh... sory delete this topic\n \n,[],Cypek,/m/cypek.2556290/,New member,"May 6, 2010",3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,False,True
38415,/t/full-video-chatt-app-like-skype.675381/,Full Video Chatt App like skype,"May 4, 2010",\nim deploying soon and was wondering if anyon...,[],pyrator,/m/pyrator.1603376/,Senior Member,"Feb 28, 2009",307,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,False,True
38420,/t/help-me-in-getting-this-app.674932/,help me in getting this app,"May 3, 2010","\nHi all,\n\ni used this(dont remember the nam...",[],chowdarygm,/m/chowdarygm.2004328/,Senior Member,"Sep 4, 2009",872,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,False,True
38435,/t/market-as-separate-app.672499/,Market as separate app,"Apr 29, 2010","\nHey people, I rebooted my phone this morning...",[],fuz1987,/m/fuz1987.2336665/,Member,"Feb 3, 2010",20,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,False,True


In [13]:
pages_nonzero_dict = pages[pages["nonzero_price"]][["url","main_developer_username", "launch_date"]].to_dict(orient="records")

In [78]:
#setting up selenium to function
edgeOption = webdriver.EdgeOptions()
edgeOption.use_chromium = True
edgeOption.add_argument("start-maximized")
edgeOption.binary_location = r"C:\Program Files (x86)\Microsoft\Edge\Application\msedge.exe"
s=service.Service(r'C:\Users\Yana Gerina\Documents\msedgedriver.exe')
driver = webdriver.Edge(service=s, options=edgeOption)

In [17]:
link_n = pages.loc[2, "url"]

In [79]:
driver.get("https://forum.xda-developers.com" + link_n)

In [80]:
for i in again:
    page = pages_nonzero_dict[i]
    driver.get("https://forum.xda-developers.com" + page["url"])
    time.sleep(3)
    #Collect the number of open and closed issues if it's available:
    for button in driver.find_elements(by=By.CLASS_NAME, value = "button-text"):
        if button.text == "Search This thread":
            button.click()
            time.sleep(2)
    input_row = driver.switch_to.active_element
    input_row.send_keys("pro |purchase|donation|paid version")
    WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CLASS_NAME, "quickSearch-ResultCloser"))).click()
    time.sleep(2)
    input_row.send_keys(Keys.TAB + Keys.TAB)
    time.sleep(2)
    member_input_row = driver.switch_to.active_element
    member_input_row.send_keys(page["main_developer_username"])
    time.sleep(2)
    member_input_row.send_keys(Keys.ENTER + Keys.ENTER)
    time.sleep(2)
    
    results  = []
    for result in driver.find_elements(by=By.CLASS_NAME, value = "contentRow-main"):
        title = result.find_element(by=By.CLASS_NAME, value = "contentRow-title")
        if re.search("Post", title.text) is not None:
            datestamp = result.find_element(by=By.CLASS_NAME, value = "u-dt").text
            post_number = [x.text for x in result.find_elements(by=By.TAG_NAME, value = "li") if re.search("Post", x.text) is not None][0].replace(",","")
            post_number = int(re.search(r"[\d,]+", post_number).group(0))
            mention = {"title": title.text, "datestamp": datestamp, "post_number": post_number}
            results.append(mention)
    time.sleep(2)
    while True:
        next_page_btn = driver.find_elements(by=By.LINK_TEXT, value = "Next")
        if len(next_page_btn) < 1:
           
            break
        else:
            next_page = driver.find_element(by=By.CLASS_NAME, value = "pageNav-jump.pageNav-jump--next").get_attribute("href")
            driver.get(next_page)
            time.sleep(2)
            for result in driver.find_elements(by=By.CLASS_NAME, value = "contentRow-main"):
                title = result.find_element(by=By.CLASS_NAME, value = "contentRow-title")
                if re.search("Post", title.text) is not None:
                    datestamp = result.find_element(by=By.CLASS_NAME, value = "u-dt").text
                    post_number = [x.text for x in result.find_elements(by=By.TAG_NAME, value = "li") if re.search("Post", x.text) is not None][0].replace(",","")
                    post_number = int(re.search(r"[\d,]+", post_number).group(0))
                    mention = {"title": title.text, "datestamp": datestamp, "post_number": post_number}
                    results.append(mention)
    if results:
        earliest_post = min(results, key=lambda x:x['post_number'])
        page["penetration_datestamp"] = earliest_post["datestamp"]
    else: 
        page["penetration_datestamp"] = np.nan
driver.quit()    


In [77]:
driver.quit()

In [86]:
scraped.to_csv("data/penetration_stamps.csv")

# Collect mentions of product in other forums after the date

## Creating a title query

In [87]:
scraped

,url,main_developer_username,launch_date,penetration_datestamp
0,/t/app-all-root-solutions-6-0-viper4android-fx...,Team_DeWitt,"Apr 7, 2018","Apr 10, 2018"
1,/t/app-6-0-netguard-no-root-firewall.3233012/,M66B,"Oct 25, 2015","Oct 31, 2015"
2,/t/app-4-0-3-4-1-rotation-orientation-manager-...,Pranav Pandey,"Jul 21, 2014","Jan 29, 2015"
3,/t/app-4-4-bromite-chromium-plus-adblocking.36...,csagan5,"Oct 11, 2017","Oct 18, 2017"
4,/t/app-mod-pixel-launcher.4442077/,iamlooper,"May 6, 2022",NaN
...,...,...,...,...
1051,/t/market-enabler-paid-apps-in-any-country.786...,vineet.rox,"Sep 19, 2010",NaN
1052,/t/test-my-android-application-should-be-able-...,surajprakash,"Sep 9, 2010",NaN
1053,/t/xda-application-apk-required.746943/,aqeelhaider,"Aug 6, 2010",NaN
1054,/t/app-live-wallpaper-creator-free.748566/,potter1984,"Aug 8, 2010",NaN


In [357]:
pages_date = pd.merge(pages, scraped, on=(["url", "main_developer_username", "launch_date"]))

In [358]:
pages_date["launch_date"] = pages_date["launch_date"].str.replace("Today", "May 19, 2022")

In [359]:
pages_date["launch_date"] = pages_date["launch_date"].str.replace("Yesterday", "May 18, 2022")

In [360]:
pages_date["launch_date"] = pages_date["launch_date"].str.replace(",", "")

In [361]:
pages_date["launch_date"] = pd.to_datetime(pages_date["launch_date"], format = "%b %d %Y")

In [362]:
pages_date

,url,title,launch_date,launch_message,external_links,main_developer_username,main_developer_page,main_developer_title,main_developer_joined,main_developer_messages,...,contrlink_2,contrlink_3,contrlink_4,contrlink_5,contrlink_6,contrlink_7,number_of_developers,nonzero_price,ext_links_ind,penetration_datestamp
0,/t/app-all-root-solutions-6-0-viper4android-fx...,[APP][ALL ROOT SOLUTIONS][6.0+]ViPER4Android F...,2018-04-07,\nViPER4Android FX\n\n\n\n\n\nBy pittvandewitt...,['https://labs.xda-developers.com/store/app/co...,Team_DeWitt,/m/team_dewitt.9073985/,Senior Member,"Apr 7, 2018",71,...,https://forum.xda-developers.com/member.php?u=...,https://forum.xda-developers.com/member.php?u=...,NaN,NaN,NaN,NaN,3.0,True,True,"Apr 10, 2018"
1,/t/app-6-0-netguard-no-root-firewall.3233012/,[APP][6.0+] NetGuard - No-root firewall,2015-10-25,\n\n\n\n\n\n\nNetGuard provides simple and adv...,['https://contact.faircode.eu/?product=netguar...,M66B,/m/m66b.2799345/,Recognized Developer,"Aug 1, 2010","25,652",...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,True,True,"Oct 31, 2015"
2,/t/app-4-0-3-4-1-rotation-orientation-manager-...,[APP] [4.0.3/4.1+] Rotation | Orientation Mana...,2014-07-21,\nWith support for Android 12​Built with Mater...,"['https://material.io', 'None', 'None', 'http:...",Pranav Pandey,/m/pranav-pandey.3962236/,Recognized Developer,"Apr 15, 2011",492,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,True,True,"Jan 29, 2015"
3,/t/app-4-4-bromite-chromium-plus-adblocking.36...,[APP] [4.4+] Bromite - Chromium plus adblocking,2017-10-11,\nI have started an open source project to sup...,"['https://www.bromite.org/', 'https://forum.xd...",csagan5,/m/csagan5.8627832/,Senior Member,"Oct 11, 2017",442,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,True,True,"Oct 18, 2017"
4,/t/app-mod-pixel-launcher.4442077/,[APP][MOD] Pixel Launcher,2022-05-06,\n\n\n\n\nMod Pixel Launcher • Android 12 • An...,"['http://t.me/modulesrepo', 'https://telegra.p...",iamlooper,/m/iamlooper.11245295/,Senior Member,"Nov 14, 2020",62,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,True,True,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
922,/t/app-auto-bluetooth.774963/,[APP] Auto Bluetooth,2010-09-05,\nUPDATE: Bug Fixed and some cosmetic makeover...,[],appelflap,/m/appelflap.827262/,Inactive Recognized Developer,"Feb 9, 2008","4,204",...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,True,True,NaN
923,/t/market-enabler-paid-apps-in-any-country.786...,Market enabler!! Paid apps in any country!!,2010-09-19,\nNow u can buy paid apps from any country aro...,[],vineet.rox,/m/vineet-rox.2632378/,Member,"Jun 6, 2010",13,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,True,True,NaN
924,/t/test-my-android-application-should-be-able-...,Test my Android Application - Should be able t...,2010-09-09,\nIf you have an Android phone (1.6 and above)...,[],surajprakash,/m/surajprakash.1197157/,Member,"Sep 7, 2008",11,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,True,True,NaN
925,/t/xda-application-apk-required.746943/,XDA Application APK Required,2010-08-06,"\nHi Guys,\n\nI would appreciate if some one c...",[],aqeelhaider,/m/aqeelhaider.285789/,Senior Member,"Mar 4, 2006",76,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,True,True,NaN


In [363]:
pages_date["title"]

0      [APP][ALL ROOT SOLUTIONS][6.0+]ViPER4Android F...
1                [APP][6.0+] NetGuard - No-root firewall
2      [APP] [4.0.3/4.1+] Rotation | Orientation Mana...
3        [APP] [4.4+] Bromite - Chromium plus adblocking
4                              [APP][MOD] Pixel Launcher
                             ...                        
922                                 [APP] Auto Bluetooth
923          Market enabler!! Paid apps in any country!!
924    Test my Android Application - Should be able t...
925                         XDA Application APK Required
926              [APP] - Live Wallpaper Creator - FREE -
Name: title, Length: 927, dtype: object

In [364]:
pages_date.loc[2,"title"]

'[APP] [4.0.3/4.1+] Rotation | Orientation Manager [24.4.1] [08/04/2022]'

In [365]:
pages_date["search_query"] = pages_date["title"].str.replace(r'\[.+?\]','').str.strip().str.replace('  ', " ")

C:\Users\YANAGE~1\AppData\Local\Temp/ipykernel_21300/620787095.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  pages_date["search_query"] = pages_date["title"].str.replace(r'\[.+?\]','').str.strip().str.replace('  ', " ")


In [366]:
pages_date["search_query"]=pages_date["search_query"].str.extract(r'([A-Z][^ -!\?\.,:;]+[ A-Za-z]*)[ -\!\?\.,:;]*')

In [367]:
pages_date["search_query"] = pages_date["search_query"].str.strip()

In [368]:
pages_date

,url,title,launch_date,launch_message,external_links,main_developer_username,main_developer_page,main_developer_title,main_developer_joined,main_developer_messages,...,contrlink_3,contrlink_4,contrlink_5,contrlink_6,contrlink_7,number_of_developers,nonzero_price,ext_links_ind,penetration_datestamp,search_query
0,/t/app-all-root-solutions-6-0-viper4android-fx...,[APP][ALL ROOT SOLUTIONS][6.0+]ViPER4Android F...,2018-04-07,\nViPER4Android FX\n\n\n\n\n\nBy pittvandewitt...,['https://labs.xda-developers.com/store/app/co...,Team_DeWitt,/m/team_dewitt.9073985/,Senior Member,"Apr 7, 2018",71,...,https://forum.xda-developers.com/member.php?u=...,NaN,NaN,NaN,NaN,3.0,True,True,"Apr 10, 2018",ViPER4Android FX
1,/t/app-6-0-netguard-no-root-firewall.3233012/,[APP][6.0+] NetGuard - No-root firewall,2015-10-25,\n\n\n\n\n\n\nNetGuard provides simple and adv...,['https://contact.faircode.eu/?product=netguar...,M66B,/m/m66b.2799345/,Recognized Developer,"Aug 1, 2010","25,652",...,NaN,NaN,NaN,NaN,NaN,1.0,True,True,"Oct 31, 2015",NetGuard
2,/t/app-4-0-3-4-1-rotation-orientation-manager-...,[APP] [4.0.3/4.1+] Rotation | Orientation Mana...,2014-07-21,\nWith support for Android 12​Built with Mater...,"['https://material.io', 'None', 'None', 'http:...",Pranav Pandey,/m/pranav-pandey.3962236/,Recognized Developer,"Apr 15, 2011",492,...,NaN,NaN,NaN,NaN,NaN,1.0,True,True,"Jan 29, 2015",Rotation
3,/t/app-4-4-bromite-chromium-plus-adblocking.36...,[APP] [4.4+] Bromite - Chromium plus adblocking,2017-10-11,\nI have started an open source project to sup...,"['https://www.bromite.org/', 'https://forum.xd...",csagan5,/m/csagan5.8627832/,Senior Member,"Oct 11, 2017",442,...,NaN,NaN,NaN,NaN,NaN,1.0,True,True,"Oct 18, 2017",Bromite
4,/t/app-mod-pixel-launcher.4442077/,[APP][MOD] Pixel Launcher,2022-05-06,\n\n\n\n\nMod Pixel Launcher • Android 12 • An...,"['http://t.me/modulesrepo', 'https://telegra.p...",iamlooper,/m/iamlooper.11245295/,Senior Member,"Nov 14, 2020",62,...,NaN,NaN,NaN,NaN,NaN,1.0,True,True,NaN,Pixel Launcher
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
922,/t/app-auto-bluetooth.774963/,[APP] Auto Bluetooth,2010-09-05,\nUPDATE: Bug Fixed and some cosmetic makeover...,[],appelflap,/m/appelflap.827262/,Inactive Recognized Developer,"Feb 9, 2008","4,204",...,NaN,NaN,NaN,NaN,NaN,1.0,True,True,NaN,Auto Bluetooth
923,/t/market-enabler-paid-apps-in-any-country.786...,Market enabler!! Paid apps in any country!!,2010-09-19,\nNow u can buy paid apps from any country aro...,[],vineet.rox,/m/vineet-rox.2632378/,Member,"Jun 6, 2010",13,...,NaN,NaN,NaN,NaN,NaN,1.0,True,True,NaN,Market enabler
924,/t/test-my-android-application-should-be-able-...,Test my Android Application - Should be able t...,2010-09-09,\nIf you have an Android phone (1.6 and above)...,[],surajprakash,/m/surajprakash.1197157/,Member,"Sep 7, 2008",11,...,NaN,NaN,NaN,NaN,NaN,1.0,True,True,NaN,Test my Android Application
925,/t/xda-application-apk-required.746943/,XDA Application APK Required,2010-08-06,"\nHi Guys,\n\nI would appreciate if some one c...",[],aqeelhaider,/m/aqeelhaider.285789/,Senior Member,"Mar 4, 2006",76,...,NaN,NaN,NaN,NaN,NaN,1.0,True,True,NaN,XDA Application APK Required


In [374]:
pages_date.loc[2,"search_query"] = "Rotation Orientation Manager"

## Search of title in external threads

In [369]:
query_dict = pages_date[["search_query", "main_developer_username","url", "launch_date", "title"]].to_dict(orient="records")

In [488]:
#setting up selenium to function
edgeOption = webdriver.EdgeOptions()
edgeOption.use_chromium = True
edgeOption.add_argument("start-maximized")
edgeOption.binary_location = r"C:\Program Files (x86)\Microsoft\Edge\Application\msedge.exe"
s=service.Service(r'C:\Users\Yana Gerina\Documents\msedgedriver.exe')
driver = webdriver.Edge(service=s, options=edgeOption)

In [489]:
driver.get("https://forum.xda-developers.com/search/")

In [497]:
for query in query_dict[841:]:
    driver.get("https://forum.xda-developers.com/search/")
    if driver.find_element(By.TAG_NAME, "p").text =='The forums are currently offline while we are performing system maintenance. Be back soon!':
        time.sleep(300)
        driver.get("https://forum.xda-developers.com/search/")
    time.sleep(2)
    input_row = driver.switch_to.active_element
    input_row.send_keys(query["search_query"])
    #WebDriverWait(driver, 50).until(EC.element_to_be_clickable((By.CLASS_NAME, "quickSearch-ResultCloser"))).click()
    #time.sleep(2)
    input_row.send_keys(Keys.TAB*5)
    time.sleep(2)
    
    input_row_date = driver.switch_to.active_element
    input_row_date.send_keys(query["launch_date"].strftime("%Y-%m-%d"))
    time.sleep(2)
    input_row_date.send_keys(Keys.ENTER)
    time.sleep(2)
    driver.find_elements(By.CLASS_NAME,"button-text")[7].click()
    
    time.sleep(2)
    
    mentions = []
    for result in driver.find_elements(by=By.CLASS_NAME, value = "contentRow-main"):
            title = result.find_element(by=By.CLASS_NAME, value = "contentRow-title")
            if title.text != query["title"]:
                if re.search("Post", title.text) is not None:
                    datestamp = result.find_element(by=By.CLASS_NAME, value = "u-dt").text
                    #post_number = [x.text for x in result.find_elements(by=By.TAG_NAME, value = "li") if re.search("Post", x.text) is not None][0].replace(",","")
                    #post_number = int(re.search(r"[\d,]+", post_number).group(0))
                    author = result.find_element(By.CLASS_NAME, value = "username").text
                    if len(result.find_elements(By.TAG_NAME, "li")[0].find_elements(By.TAG_NAME, "a"))>1:
                        author_link = result.find_elements(By.TAG_NAME, "li")[0].find_element(By.TAG_NAME, "a").get_attribute("href")                            
                    post = {"title": title.text, "datestamp": datestamp, "mention_type": "post", "author": author, "author_link": author_link}
                    mentions.append(post.copy())

                if re.search("Thread", title.text) is not None:
                    datestamp = result.find_element(by=By.CLASS_NAME, value = "u-dt").text
                    if len(result.find_elements(By.TAG_NAME, "li")[0].find_elements(By.TAG_NAME, "a"))>1:
                        author_link = result.find_elements(By.TAG_NAME, "li")[0].find_element(By.TAG_NAME, "a").get_attribute("href")                                                       
                    author = result.find_element(By.CLASS_NAME, value = "username").text
                    replies = result.find_elements(By.TAG_NAME, value = "li")[3].text
                    thread = {"title": title.text, "datestamp": datestamp, "mention_type": "thread", "author": author, "author_link": author_link, "replies": replies}
                    mentions.append(thread.copy())
    while True:
        next_page_btn = driver.find_elements(by=By.LINK_TEXT, value = "Next")
        if len(next_page_btn) < 1:

            break
        else:
            next_page = driver.find_element(by=By.CLASS_NAME, value = "pageNav-jump.pageNav-jump--next").get_attribute("href")
            driver.get(next_page)
            time.sleep(2)
            for result in driver.find_elements(by=By.CLASS_NAME, value = "contentRow-main"):
                title = result.find_element(by=By.CLASS_NAME, value = "contentRow-title")
                if title.text != query_dict[0]["title"]:
                    if re.search("Post", title.text) is not None:
                        datestamp = result.find_element(by=By.CLASS_NAME, value = "u-dt").text
                        #post_number = [x.text for x in result.find_elements(by=By.TAG_NAME, value = "li") if re.search("Post", x.text) is not None][0].replace(",","")
                        #post_number = int(re.search(r"[\d,]+", post_number).group(0))
                        author = result.find_element(By.CLASS_NAME, value = "username").text
                        if len(result.find_elements(By.TAG_NAME, "li")[0].find_elements(By.TAG_NAME, "a"))>1:
                            author_link = result.find_elements(By.TAG_NAME, "li")[0].find_element(By.TAG_NAME, "a").get_attribute("href")                            
                        post = {"title": title.text, "datestamp": datestamp, "mention_type": "post", "author": author, "author_link": author_link}
                        mentions.append(post.copy())

                    if re.search("Thread", title.text) is not None:
                        datestamp = result.find_element(by=By.CLASS_NAME, value = "u-dt").text
                        if len(result.find_elements(By.TAG_NAME, "li")[0].find_elements(By.TAG_NAME, "a"))>1:
                            author_link = result.find_elements(By.TAG_NAME, "li")[0].find_element(By.TAG_NAME, "a").get_attribute("href")                            
                        author = result.find_element(By.CLASS_NAME, value = "username").text
                        replies = result.find_elements(By.TAG_NAME, value = "li")[3].text
                        thread = {"title": title.text, "datestamp": datestamp, "mention_type": "thread", "author": author, "author_link": author_link, "replies": replies}
                        mentions.append(thread.copy())
                

    query["mentions"] = mentions.copy()
    query["n_threads"] = len([d for d in mentions if d['mention_type'] == 'thread'])
    query["n_posts"] = len([d for d in mentions if d['mention_type'] == 'post'])
driver.quit()

In [445]:
driver.find_element(By.TAG_NAME, "p").text =='The forums are currently offline while we are performing system maintenance. Be back soon!'

True

In [498]:
df = pd.DataFrame(query_dict)

In [503]:
df

,search_query,main_developer_username,url,launch_date,title,mentions,n_threads,n_posts
0,ViPER4Android FX,Team_DeWitt,/t/app-all-root-solutions-6-0-viper4android-fx...,2018-04-07,[APP][ALL ROOT SOLUTIONS][6.0+]ViPER4Android F...,[{'title': 'Thread [APP][ALL ROOT SOLUTIONS][6...,169.0,835.0
1,NetGuard,M66B,/t/app-6-0-netguard-no-root-firewall.3233012/,2015-10-25,[APP][6.0+] NetGuard - No-root firewall,[{'title': 'Thread WiFi Problems with Android ...,78.0,913.0
2,Rotation Orientation Manager,Pranav Pandey,/t/app-4-0-3-4-1-rotation-orientation-manager-...,2014-07-21,[APP] [4.0.3/4.1+] Rotation | Orientation Mana...,[{'title': 'Thread SP9820E SoC powered kid wat...,92.0,451.0
3,Bromite,csagan5,/t/app-4-4-bromite-chromium-plus-adblocking.36...,2017-10-11,[APP] [4.4+] Bromite - Chromium plus adblocking,[{'title': 'Thread [MAGISK] [MODULE] Webview M...,126.0,869.0
4,Pixel Launcher,iamlooper,/t/app-mod-pixel-launcher.4442077/,2022-05-06,[APP][MOD] Pixel Launcher,"[{'title': 'Thread [APP][MOD] Pixel Launcher',...",25.0,187.0
...,...,...,...,...,...,...,...,...
922,Auto Bluetooth,appelflap,/t/app-auto-bluetooth.774963/,2010-09-05,[APP] Auto Bluetooth,[{'title': 'Thread [ROOT ONLY] AA AIO TWEAKER ...,999.0,1.0
923,Market enabler,vineet.rox,/t/market-enabler-paid-apps-in-any-country.786...,2010-09-19,Market enabler!! Paid apps in any country!!,"[{'title': 'Thread project SLIMBOX', 'datestam...",599.0,2.0
924,Test my Android Application,surajprakash,/t/test-my-android-application-should-be-able-...,2010-09-09,Test my Android Application - Should be able t...,[],0.0,0.0
925,XDA Application APK Required,aqeelhaider,/t/xda-application-apk-required.746943/,2010-08-06,XDA Application APK Required,[{'title': 'Thread Your Google Play Store is b...,730.0,263.0


In [502]:
df.to_csv("data/first_N.csv", sep=";")

In [499]:
df[df["mentions"].isnull()==True][2:]

,search_query,main_developer_username,url,launch_date,title,mentions,n_threads,n_posts
451,Facebook v,elesbb,/t/app-mod-update-12-26-14-facebook-v23-0-with...,2013-06-21,[APP][MOD] [UPDATE: 12-26-14] Facebook v23.0 w...,NaN,NaN,NaN
474,Apps,rojoloco47,/t/200-apps-games-source-codes-for-sale-100.32...,2015-10-17,200+ Apps / Games source codes for sale | $100,NaN,NaN,NaN
493,Discussion,digitalhigh,/t/app-mod-tut-discussion-plex-w-android-tv-ui...,2015-01-13,[APP][MOD][TUT]Discussion: Plex w. Android TV ...,NaN,NaN,NaN
494,Bike,nazarsokol,/t/app-free-4-0-bike-3d-configurator-1-2.3016915/,2015-01-30,[APP][FREE][4.0.+] Bike 3d Configurator 1.2,NaN,NaN,NaN


In [486]:
driver.quit()

## Merge with first batch data

In [588]:
# Load the latest data
gp_data = pd.read_csv('data/full_gp_data.csv', sep=";", index_col = "Unnamed: 0") 

In [547]:
#df = df.drop('search_query',axis=1)

KeyError: "['search_query'] not found in axis"

In [651]:
done = gp_data.merge(df, on = ["url","title","launch_date"], how="inner")

In [652]:
done

,gp_url,url,app_name,launch_date,title,search_query,main_developer_username,mentions,n_threads,n_posts
0,play.google.com/store/apps/details?id=eu.fairc...,/t/app-6-0-netguard-no-root-firewall.3233012/,NetGuard - no-root firewall,2015-10-25,[APP][6.0+] NetGuard - No-root firewall,NetGuard&noroot&firewall,M66B,[{'title': 'Thread WiFi Problems with Android ...,78.0,913.0
1,play.google.com/store/apps/details?id=com.pran...,/t/app-4-0-3-4-1-rotation-orientation-manager-...,Rotation - Orientation Manager,2014-07-21,[APP] [4.0.3/4.1+] Rotation | Orientation Mana...,Rotation&Orientation&Manager,Pranav Pandey,[{'title': 'Thread SP9820E SoC powered kid wat...,92.0,451.0
2,play.google.com/store/apps/details?id=flar2.de...,/t/app-devcheck-1-19-system-information.3171427/,DevCheck Hardware and System Info,2015-08-03,[APP] DevCheck 1.19 - System Information,DevCheck&Hardware&and&System&Info,flar2,[{'title': 'Thread Root SWM-8802 / Alps CAR_G2...,93.0,917.0
3,play.google.com/store/apps/details?id=com.page...,/t/app-5-0-batteryguru-monitor-saver.3980767/,Battery Guru - Battery Monitor - Battery Saver,2019-10-10,[APP][5.0+] BatteryGuru: Monitor & Saver,Battery&Guru&Battery&Monitor&Battery&Saver,Paget96,[{'title': 'Post Question Why is my phone runn...,0.0,229.0
4,play.google.com/store/apps/details?id=com.teqt...,/t/app-teqtic-lock-me-out-freedom-from-phone-a...,Lock Me Out 🥇 App blocker & website blocker,2014-09-24,[APP][TEQTIC] Lock Me Out - Freedom from phone...,Lock&Me&Out&App&blocker&&website&blocker,Flyview,[{'title': 'Thread [APP][v2.8][20/03]NiceLock ...,1000.0,0.0
...,...,...,...,...,...,...,...,...,...,...
149,play.google.com/store/apps/details?id=com.doub...,/t/app-4-1-doubletwist-cloudplayer-gets-cloud-...,CloudPlayer™ by doubleTwist cloud & offline pl...,2016-05-11,[APP][4.1+] doubleTwist CloudPlayer gets cloud...,CloudPlayer&by&doubleTwist&cloud&&offline&player,twisterdroid,[],0.0,0.0
150,play.google.com/store/apps/details?id=com.smar...,/t/app-root-5-0-smartpack-kernel-manager.3854717/,SmartPack-Kernel Manager,2018-10-16,[App][Root][5.0+] SmartPack-Kernel Manager,SmartPackKernel&Manager,sunilpaulmathew,[{'title': 'Thread [M.O.D. KERNEL Q 1.5] MIUI ...,78.0,912.0
151,play.google.com/store/apps/details?id=com.anan...,/t/app-4-3-notific-lockscreen-notification-1-j...,Notific Pro,2014-07-04,[APP][4.3+] Notific : Lockscreen Notification ...,Notific&Pro,anandbibek,"[{'title': 'Thread [APP][1.3] Tap, Tap: Double...",990.0,0.0
152,play.google.com/store/apps/details?id=com.delp...,/t/flud-torrent-downloader-best-app-for-u-reco...,Flud - Torrent Downloader,2015-03-26,Flud - Torrent Downloader [Best App For U] [ R...,Flud&Torrent&Downloader,naresh9,[{'title': 'Thread download torrents to a usb ...,37.0,282.0


In [626]:
gp_data["search_query"]=gp_data["app_name"].str.replace("  "," ").str.replace(" ", "&")

In [627]:
gp_data["search_query"]=gp_data["search_query"].str.replace("[^0-9A-Za-z\&]", "").str.replace("\&\&", "&")

C:\Users\YANAGE~1\AppData\Local\Temp/ipykernel_21300/417726274.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  gp_data["search_query"]=gp_data["search_query"].str.replace("[^0-9A-Za-z\&]", "").str.replace("\&\&", "&")


In [592]:
#done_dict = done[["gp_url","url","search_query"]].to_dict(orient="records")

In [593]:
gp_data = gp_data[gp_data["url"].isnull()==False]

In [594]:
gp_data["launch_date"] = gp_data["launch_date"].str.replace("Today", "May 19, 2022")

In [595]:
gp_data["launch_date"] = gp_data["launch_date"].str.replace("Yesterday", "May 18, 2022")

In [596]:
gp_data["launch_date"] = gp_data["launch_date"].str.replace(",", "")

In [597]:
#gp_data[gp_data["launch_date"].str.contains("at")==True]

In [598]:
gp_data["launch_date"] = gp_data["launch_date"].str.replace(" at .+","")

C:\Users\YANAGE~1\AppData\Local\Temp/ipykernel_21300/4055325791.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  gp_data["launch_date"] = gp_data["launch_date"].str.replace(" at .+","")


In [599]:
gp_data["launch_date"] = pd.to_datetime(gp_data["launch_date"], format = "%b %d %Y")

In [616]:
gp_data

,gp_url,url,app_name,launch_date,title,search_query
0,play.google.com/store/apps/details?id=com.baiu...,/t/app-6-0-compass-of-chance.4440211/,Compass of Chance: Your Dream,2022-05-02,[APP][6.0+]Compass of Chance,Compass&of&Chance&Your&Dream
1,play.google.com/store/apps/details?id=eu.fairc...,/t/app-6-0-netguard-no-root-firewall.3233012/,NetGuard - no-root firewall,2015-10-25,[APP][6.0+] NetGuard - No-root firewall,NetGuard&&noroot&firewall
2,play.google.com/store/apps/details?id=com.helg...,/t/app-11-dual-camera-video-recorder.4447581/,Dual Camera Video Recorder,2022-05-19,[APP][11+] Dual Camera Video Recorder,Dual&Camera&Video&Recorder
4,play.google.com/store/apps/details?id=rs.pedja...,/t/app-root-4-0-battery-charge-limit-v1-1-1.35...,[ROOT] aLogcat,2017-02-15,[APP][ROOT][4.0+]Battery Charge Limit v1.1.1,ROOT&aLogcat
5,play.google.com/store/apps/details?id=rs.pedja...,/t/app-4-1-20140920-alogcat-root-v1-0-1.2876985/,[ROOT] aLogcat,2014-09-14,[APP][4.1+][20140920] aLogcat ROOT - v1.0.1,ROOT&aLogcat
...,...,...,...,...,...,...
3416,play.google.com/store/apps/details?id=com.sien...,/t/free-app-ur-launcher.2924469/,UR 3D Launcher,2014-10-31,[FREE][APP] UR launcher,UR&3D&Launcher
3424,play.google.com/store/apps/details?id=christma...,/t/christmas-ringtones-app.2003588/,Weihnachten Klingeltöne,2012-11-20,Christmas Ringtones [APP],Weihnachten&Klingeltne
3425,play.google.com/store/apps/details?id=ua.maksd...,/t/app-1-6-horoscopes-d_t.1902068/,Horoscope for Tomorrow,2012-09-24,[APP][1.6+] Horoscopes D_T,Horoscope&for&Tomorrow
3428,play.google.com/store/apps/details?id=com.alen...,/t/app-mirror-miui-gallery-and-quickpic-pretty...,Galería de fotos - QuickPic,2012-04-05,[APP-MIRROR] MIUI Gallery and QuickPic (Pretty...,Galera&de&fotos&&QuickPic


In [628]:
gp_data_new = gp_data[~(gp_data.url.isin(done['url']))]

In [629]:
gp_data_new

,gp_url,url,app_name,launch_date,title,search_query
0,play.google.com/store/apps/details?id=com.baiu...,/t/app-6-0-compass-of-chance.4440211/,Compass of Chance: Your Dream,2022-05-02,[APP][6.0+]Compass of Chance,Compass&of&Chance&Your&Dream
2,play.google.com/store/apps/details?id=com.helg...,/t/app-11-dual-camera-video-recorder.4447581/,Dual Camera Video Recorder,2022-05-19,[APP][11+] Dual Camera Video Recorder,Dual&Camera&Video&Recorder
4,play.google.com/store/apps/details?id=rs.pedja...,/t/app-root-4-0-battery-charge-limit-v1-1-1.35...,[ROOT] aLogcat,2017-02-15,[APP][ROOT][4.0+]Battery Charge Limit v1.1.1,ROOT&aLogcat
5,play.google.com/store/apps/details?id=rs.pedja...,/t/app-4-1-20140920-alogcat-root-v1-0-1.2876985/,[ROOT] aLogcat,2014-09-14,[APP][4.1+][20140920] aLogcat ROOT - v1.0.1,ROOT&aLogcat
6,play.google.com/store/apps/details?id=com.morn...,/t/app-free-4-1-warm-up-exercises-and-training...,Warm Up Exercises and training,2022-05-18,[app][Free][4.1+] Warm Up Exercises and training,Warm&Up&Exercises&and&training
...,...,...,...,...,...,...
3416,play.google.com/store/apps/details?id=com.sien...,/t/free-app-ur-launcher.2924469/,UR 3D Launcher,2014-10-31,[FREE][APP] UR launcher,UR&3D&Launcher
3424,play.google.com/store/apps/details?id=christma...,/t/christmas-ringtones-app.2003588/,Weihnachten Klingeltöne,2012-11-20,Christmas Ringtones [APP],Weihnachten&Klingeltne
3425,play.google.com/store/apps/details?id=ua.maksd...,/t/app-1-6-horoscopes-d_t.1902068/,Horoscope for Tomorrow,2012-09-24,[APP][1.6+] Horoscopes D_T,Horoscope&for&Tomorrow
3428,play.google.com/store/apps/details?id=com.alen...,/t/app-mirror-miui-gallery-and-quickpic-pretty...,Galería de fotos - QuickPic,2012-04-05,[APP-MIRROR] MIUI Gallery and QuickPic (Pretty...,Galera&de&fotos&QuickPic


In [630]:
gp_data_dict = gp_data_new.to_dict(orient="records")

In [635]:
#setting up selenium to function
edgeOption = webdriver.EdgeOptions()
edgeOption.use_chromium = True
edgeOption.add_argument("start-maximized")
edgeOption.binary_location = r"C:\Program Files (x86)\Microsoft\Edge\Application\msedge.exe"
s=service.Service(r'C:\Users\Yana Gerina\Documents\msedgedriver.exe')
driver = webdriver.Edge(service=s, options=edgeOption)

In [636]:
driver.get("https://forum.xda-developers.com/search/")

In [637]:
for query in gp_data_dict[:95]:
    driver.get("https://forum.xda-developers.com/search/")
    if driver.find_element(By.TAG_NAME, "p").text =='The forums are currently offline while we are performing system maintenance. Be back soon!':
        time.sleep(300)
        driver.get("https://forum.xda-developers.com/search/")
    time.sleep(2)
    input_row = driver.switch_to.active_element
    input_row.send_keys(query["search_query"])
    #WebDriverWait(driver, 50).until(EC.element_to_be_clickable((By.CLASS_NAME, "quickSearch-ResultCloser"))).click()
    #time.sleep(2)
    input_row.send_keys(Keys.TAB*5)
    time.sleep(2)
    
    input_row_date = driver.switch_to.active_element
    input_row_date.send_keys(query["launch_date"].strftime("%Y-%m-%d"))
    time.sleep(2)
    input_row_date.send_keys(Keys.ENTER)
    time.sleep(2)
    driver.find_elements(By.CLASS_NAME,"button-text")[7].click()
    
    time.sleep(2)
    
    mentions = []
    for result in driver.find_elements(by=By.CLASS_NAME, value = "contentRow-main"):
            title = result.find_element(by=By.CLASS_NAME, value = "contentRow-title")
            if title.text != query["title"]:
                if re.search("Post", title.text) is not None:
                    datestamp = result.find_element(by=By.CLASS_NAME, value = "u-dt").text
                    #post_number = [x.text for x in result.find_elements(by=By.TAG_NAME, value = "li") if re.search("Post", x.text) is not None][0].replace(",","")
                    #post_number = int(re.search(r"[\d,]+", post_number).group(0))
                    author = result.find_element(By.CLASS_NAME, value = "username").text
                    if len(result.find_elements(By.TAG_NAME, "li")[0].find_elements(By.TAG_NAME, "a"))>1:
                        author_link = result.find_elements(By.TAG_NAME, "li")[0].find_element(By.TAG_NAME, "a").get_attribute("href")                            
                    post = {"title": title.text, "datestamp": datestamp, "mention_type": "post", "author": author, "author_link": author_link}
                    mentions.append(post.copy())

                if re.search("Thread", title.text) is not None:
                    datestamp = result.find_element(by=By.CLASS_NAME, value = "u-dt").text
                    if len(result.find_elements(By.TAG_NAME, "li")[0].find_elements(By.TAG_NAME, "a"))>1:
                        author_link = result.find_elements(By.TAG_NAME, "li")[0].find_element(By.TAG_NAME, "a").get_attribute("href")                                                       
                    author = result.find_element(By.CLASS_NAME, value = "username").text
                    replies = result.find_elements(By.TAG_NAME, value = "li")[3].text
                    thread = {"title": title.text, "datestamp": datestamp, "mention_type": "thread", "author": author, "author_link": author_link, "replies": replies}
                    mentions.append(thread.copy())
    while True:
        next_page_btn = driver.find_elements(by=By.LINK_TEXT, value = "Next")
        if len(next_page_btn) < 1:

            break
        else:
            next_page = driver.find_element(by=By.CLASS_NAME, value = "pageNav-jump.pageNav-jump--next").get_attribute("href")
            driver.get(next_page)
            time.sleep(2)
            for result in driver.find_elements(by=By.CLASS_NAME, value = "contentRow-main"):
                title = result.find_element(by=By.CLASS_NAME, value = "contentRow-title")
                if title.text != query_dict[0]["title"]:
                    if re.search("Post", title.text) is not None:
                        datestamp = result.find_element(by=By.CLASS_NAME, value = "u-dt").text
                        #post_number = [x.text for x in result.find_elements(by=By.TAG_NAME, value = "li") if re.search("Post", x.text) is not None][0].replace(",","")
                        #post_number = int(re.search(r"[\d,]+", post_number).group(0))
                        author = result.find_element(By.CLASS_NAME, value = "username").text
                        if len(result.find_elements(By.TAG_NAME, "li")[0].find_elements(By.TAG_NAME, "a"))>1:
                            author_link = result.find_elements(By.TAG_NAME, "li")[0].find_element(By.TAG_NAME, "a").get_attribute("href")                            
                        post = {"title": title.text, "datestamp": datestamp, "mention_type": "post", "author": author, "author_link": author_link}
                        mentions.append(post.copy())

                    if re.search("Thread", title.text) is not None:
                        datestamp = result.find_element(by=By.CLASS_NAME, value = "u-dt").text
                        if len(result.find_elements(By.TAG_NAME, "li")[0].find_elements(By.TAG_NAME, "a"))>1:
                            author_link = result.find_elements(By.TAG_NAME, "li")[0].find_element(By.TAG_NAME, "a").get_attribute("href")                            
                        author = result.find_element(By.CLASS_NAME, value = "username").text
                        replies = result.find_elements(By.TAG_NAME, value = "li")[3].text
                        thread = {"title": title.text, "datestamp": datestamp, "mention_type": "thread", "author": author, "author_link": author_link, "replies": replies}
                        mentions.append(thread.copy())
                

    query["mentions"] = mentions.copy()
    query["n_threads"] = len([d for d in mentions if d['mention_type'] == 'thread'])
    query["n_posts"] = len([d for d in mentions if d['mention_type'] == 'post'])
driver.quit()

In [583]:
driver.quit()

In [639]:
finished = pd.DataFrame(gp_data_dict)

In [641]:
finished

,gp_url,url,app_name,launch_date,title,search_query,mentions,n_threads,n_posts
0,play.google.com/store/apps/details?id=com.baiu...,/t/app-6-0-compass-of-chance.4440211/,Compass of Chance: Your Dream,2022-05-02,[APP][6.0+]Compass of Chance,Compass&of&Chance&Your&Dream,[],0,0
1,play.google.com/store/apps/details?id=com.helg...,/t/app-11-dual-camera-video-recorder.4447581/,Dual Camera Video Recorder,2022-05-19,[APP][11+] Dual Camera Video Recorder,Dual&Camera&Video&Recorder,[{'title': 'Thread Sputnik Rom S20 FE 4G Exyno...,3,4
2,play.google.com/store/apps/details?id=rs.pedja...,/t/app-root-4-0-battery-charge-limit-v1-1-1.35...,[ROOT] aLogcat,2017-02-15,[APP][ROOT][4.0+]Battery Charge Limit v1.1.1,ROOT&aLogcat,[{'title': 'Post [OFFICIAL][ZS620KL] LineageOS...,6,187
3,play.google.com/store/apps/details?id=rs.pedja...,/t/app-4-1-20140920-alogcat-root-v1-0-1.2876985/,[ROOT] aLogcat,2014-09-14,[APP][4.1+][20140920] aLogcat ROOT - v1.0.1,ROOT&aLogcat,[{'title': 'Thread Rooting an LG V20 H910 (AT&...,26,414
4,play.google.com/store/apps/details?id=com.morn...,/t/app-free-4-1-warm-up-exercises-and-training...,Warm Up Exercises and training,2022-05-18,[app][Free][4.1+] Warm Up Exercises and training,Warm&Up&Exercises&and&training,[{'title': 'Thread [app][Free][4.1+] Warm Up E...,1,0
...,...,...,...,...,...,...,...,...,...
1303,play.google.com/store/apps/details?id=com.sien...,/t/free-app-ur-launcher.2924469/,UR 3D Launcher,2014-10-31,[FREE][APP] UR launcher,UR&3D&Launcher,"[{'title': 'Post Pixel Launcher 2.0 problem', ...",1,19
1304,play.google.com/store/apps/details?id=christma...,/t/christmas-ringtones-app.2003588/,Weihnachten Klingeltöne,2012-11-20,Christmas Ringtones [APP],Weihnachten&Klingeltne,[],0,0
1305,play.google.com/store/apps/details?id=ua.maksd...,/t/app-1-6-horoscopes-d_t.1902068/,Horoscope for Tomorrow,2012-09-24,[APP][1.6+] Horoscopes D_T,Horoscope&for&Tomorrow,[],0,0
1306,play.google.com/store/apps/details?id=com.alen...,/t/app-mirror-miui-gallery-and-quickpic-pretty...,Galería de fotos - QuickPic,2012-04-05,[APP-MIRROR] MIUI Gallery and QuickPic (Pretty...,Galera&de&fotos&QuickPic,[],0,0


In [649]:
done

,gp_url,url,app_name,launch_date_x,title_x,main_developer_username,launch_date_y,title_y,mentions,n_threads,n_posts
0,play.google.com/store/apps/details?id=eu.fairc...,/t/app-6-0-netguard-no-root-firewall.3233012/,NetGuard - no-root firewall,"Oct 25, 2015",[APP][6.0+] NetGuard - No-root firewall,M66B,2015-10-25,[APP][6.0+] NetGuard - No-root firewall,[{'title': 'Thread WiFi Problems with Android ...,78.0,913.0
1,play.google.com/store/apps/details?id=com.pran...,/t/app-4-0-3-4-1-rotation-orientation-manager-...,Rotation - Orientation Manager,"Jul 21, 2014",[APP] [4.0.3/4.1+] Rotation | Orientation Mana...,Pranav Pandey,2014-07-21,[APP] [4.0.3/4.1+] Rotation | Orientation Mana...,[{'title': 'Thread SP9820E SoC powered kid wat...,92.0,451.0
2,play.google.com/store/apps/details?id=flar2.de...,/t/app-devcheck-1-19-system-information.3171427/,DevCheck Hardware and System Info,"Aug 3, 2015",[APP] DevCheck 1.19 - System Information,flar2,2015-08-03,[APP] DevCheck 1.19 - System Information,[{'title': 'Thread Root SWM-8802 / Alps CAR_G2...,93.0,917.0
3,play.google.com/store/apps/details?id=com.page...,/t/app-5-0-batteryguru-monitor-saver.3980767/,Battery Guru - Battery Monitor - Battery Saver,"Oct 10, 2019",[APP][5.0+] BatteryGuru: Monitor & Saver,Paget96,2019-10-10,[APP][5.0+] BatteryGuru: Monitor & Saver,[{'title': 'Post Question Why is my phone runn...,0.0,229.0
4,play.google.com/store/apps/details?id=com.teqt...,/t/app-teqtic-lock-me-out-freedom-from-phone-a...,Lock Me Out 🥇 App blocker & website blocker,"Sep 24, 2014",[APP][TEQTIC] Lock Me Out - Freedom from phone...,Flyview,2014-09-24,[APP][TEQTIC] Lock Me Out - Freedom from phone...,[{'title': 'Thread [APP][v2.8][20/03]NiceLock ...,1000.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
149,play.google.com/store/apps/details?id=com.doub...,/t/app-4-1-doubletwist-cloudplayer-gets-cloud-...,CloudPlayer™ by doubleTwist cloud & offline pl...,"May 11, 2016",[APP][4.1+] doubleTwist CloudPlayer gets cloud...,twisterdroid,2016-05-11,[APP][4.1+] doubleTwist CloudPlayer gets cloud...,[],0.0,0.0
150,play.google.com/store/apps/details?id=com.smar...,/t/app-root-5-0-smartpack-kernel-manager.3854717/,SmartPack-Kernel Manager,"Oct 16, 2018",[App][Root][5.0+] SmartPack-Kernel Manager,sunilpaulmathew,2018-10-16,[App][Root][5.0+] SmartPack-Kernel Manager,[{'title': 'Thread [M.O.D. KERNEL Q 1.5] MIUI ...,78.0,912.0
151,play.google.com/store/apps/details?id=com.anan...,/t/app-4-3-notific-lockscreen-notification-1-j...,Notific Pro,"Jul 4, 2014",[APP][4.3+] Notific : Lockscreen Notification ...,anandbibek,2014-07-04,[APP][4.3+] Notific : Lockscreen Notification ...,"[{'title': 'Thread [APP][1.3] Tap, Tap: Double...",990.0,0.0
152,play.google.com/store/apps/details?id=com.delp...,/t/flud-torrent-downloader-best-app-for-u-reco...,Flud - Torrent Downloader,"Mar 26, 2015",Flud - Torrent Downloader [Best App For U] [ R...,naresh9,2015-03-26,Flud - Torrent Downloader [Best App For U] [ R...,[{'title': 'Thread download torrents to a usb ...,37.0,282.0


In [653]:
data = pd.concat([finished, done]).drop_duplicates(["url"]).reset_index(drop=True)

In [654]:
data

,gp_url,url,app_name,launch_date,title,search_query,mentions,n_threads,n_posts,main_developer_username
0,play.google.com/store/apps/details?id=com.baiu...,/t/app-6-0-compass-of-chance.4440211/,Compass of Chance: Your Dream,2022-05-02,[APP][6.0+]Compass of Chance,Compass&of&Chance&Your&Dream,[],0.0,0.0,NaN
1,play.google.com/store/apps/details?id=com.helg...,/t/app-11-dual-camera-video-recorder.4447581/,Dual Camera Video Recorder,2022-05-19,[APP][11+] Dual Camera Video Recorder,Dual&Camera&Video&Recorder,[{'title': 'Thread Sputnik Rom S20 FE 4G Exyno...,3.0,4.0,NaN
2,play.google.com/store/apps/details?id=rs.pedja...,/t/app-root-4-0-battery-charge-limit-v1-1-1.35...,[ROOT] aLogcat,2017-02-15,[APP][ROOT][4.0+]Battery Charge Limit v1.1.1,ROOT&aLogcat,[{'title': 'Post [OFFICIAL][ZS620KL] LineageOS...,6.0,187.0,NaN
3,play.google.com/store/apps/details?id=rs.pedja...,/t/app-4-1-20140920-alogcat-root-v1-0-1.2876985/,[ROOT] aLogcat,2014-09-14,[APP][4.1+][20140920] aLogcat ROOT - v1.0.1,ROOT&aLogcat,[{'title': 'Thread Rooting an LG V20 H910 (AT&...,26.0,414.0,NaN
4,play.google.com/store/apps/details?id=com.morn...,/t/app-free-4-1-warm-up-exercises-and-training...,Warm Up Exercises and training,2022-05-18,[app][Free][4.1+] Warm Up Exercises and training,Warm&Up&Exercises&and&training,[{'title': 'Thread [app][Free][4.1+] Warm Up E...,1.0,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...
1457,play.google.com/store/apps/details?id=com.doub...,/t/app-4-1-doubletwist-cloudplayer-gets-cloud-...,CloudPlayer™ by doubleTwist cloud & offline pl...,2016-05-11,[APP][4.1+] doubleTwist CloudPlayer gets cloud...,CloudPlayer&by&doubleTwist&cloud&&offline&player,[],0.0,0.0,twisterdroid
1458,play.google.com/store/apps/details?id=com.smar...,/t/app-root-5-0-smartpack-kernel-manager.3854717/,SmartPack-Kernel Manager,2018-10-16,[App][Root][5.0+] SmartPack-Kernel Manager,SmartPackKernel&Manager,[{'title': 'Thread [M.O.D. KERNEL Q 1.5] MIUI ...,78.0,912.0,sunilpaulmathew
1459,play.google.com/store/apps/details?id=com.anan...,/t/app-4-3-notific-lockscreen-notification-1-j...,Notific Pro,2014-07-04,[APP][4.3+] Notific : Lockscreen Notification ...,Notific&Pro,"[{'title': 'Thread [APP][1.3] Tap, Tap: Double...",990.0,0.0,anandbibek
1460,play.google.com/store/apps/details?id=com.delp...,/t/flud-torrent-downloader-best-app-for-u-reco...,Flud - Torrent Downloader,2015-03-26,Flud - Torrent Downloader [Best App For U] [ R...,Flud&Torrent&Downloader,[{'title': 'Thread download torrents to a usb ...,37.0,282.0,naresh9


In [655]:
#output data
#data.to_csv("data/n_data.csv", sep=";")

In [661]:
data = data[data["mentions"].isnull()==False]

In [664]:
ne = data[data['mentions'].map(lambda d: len(d)) > 0]

In [670]:
max(ne["n_posts"])

960.0

In [ ]:
data[["url","mentions"]].to_dict(orient="records")